# Detect the colors of each address in a folder


Ideally this should be run in colab or a machine with good reasources.

In [ ]:
# In colab one can download all the images to a file then upload to drive and get them from from 

# google.colab import drive
# drive.mount('/content/drive') 

In [ ]:
# imports
import os
from tabulate import tabulate

## Load images from source directory:

In [ ]:
dataset = "borgundvegen1"
Source = os.listdir(f'data/buildings/{dataset}')
print(Source[0])

# Other options
options = os.listdir("data/buildings")[:4]
print(f'Other options: {options}')

Making a class and helpful functions for a better structure.

In [ ]:
class Building:
    def __init__(self, addr, pC, city_, lat, long, dir, type):
        self.address = addr
        self.postalCode = pC
        self.city = city_
        self.latitude = lat
        self.longitude = long
        self.filetype = type
        self.directions = [dir]
        self.filenames = []
        self.predictions = {
                                'North':("empty", 0),
                                'East': ("empty", 0),
                                'West': ("empty", 0),
                                'South': ("empty", 0)
                            }
        self.predicted = "None"
    
    def getTableItems(self):
        return [
                    self.address,
                    f"{self.postalCode}, {self.city}",
                    f"{self.latitude} N",
                    f"{self.longitude} E", 
                    len(self.directions),
                    self.predicted, 
                    self.predictions["North"],
                    self.predictions["East"],
                    self.predictions["West"],
                    self.predictions["South"],
                ]
    
    def addPrediction(self, filename: str, prediction:tuple):
        direction = filename.split('_')[-1].split('.')[0]
        self.predictions[direction] = prediction
    
    def MakePrediction(self):
        votes = {}
        for key, predictionTuple in self.predictions.items():
            class_, conf_ = predictionTuple
            # Choosing the one with the highest combined confidence
            if class_ == "empty":
                continue
            elif class_ not in votes:
                votes[class_] = float(conf_)
            else: 
                votes[class_] += float(conf_)
        if len(votes) == 0:
            self.predicted = ("empty", 0)
        else:
            self.predicted = max(votes, key=votes.get)
            
    def getDumpformat(self):
        return [
                    self.postalCode,
                    self.city,
                    self.latitude,
                    self.longitude,
                    self.predicted,
                    self.predictions["North"],
                    self.predictions["East"],
                    self.predictions["West"],
                    self.predictions["South"],
                ]

In [ ]:
def readInSources(Source):
    Buildings = dict()
    for filename in Source:
        filename_splitted = filename.split('_')
        
        # If the filename is not in the correct format, skip it. This means it is a building plot or something like that
        if filename_splitted[0].isnumeric(): 
            print(f"bad file encounterd: {filename}")
            print()
            continue
        # else continue 
        
        address, postalCode, city, latitude, longitude, directionAndFiletype = filename_splitted
        direction, filetype = directionAndFiletype.split('.')
        
        if address in Buildings:
            Buildings[address].directions.append(direction)
        else:
            Buildings[address] = Building(address, postalCode, city, latitude, longitude, direction, filetype)
        Buildings[address].filenames.append(f"data/buildings/{dataset}/{filename}")
    return Buildings
            

In [ ]:
def PrintStatistics(Buildings):
    header = ['Address', 'Postal address', 'Lat', 'Long', 'Directions', 'Predicted','North', 'East', 'West', 'South']

    tableContent = []
    for _, building in Buildings.items():
        tableContent.append(building.getTableItems())
    print(tabulate(tableContent, headers=header))

In [ ]:
SourceBuildings = readInSources(Source)
print("Table before runnning the model")
PrintStatistics(SourceBuildings)

## Loading the model

In [ ]:
# More imports
from IPython import display
display.clear_output()
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from IPython.display import display, Image

In [ ]:
pathToModel = "data/models/v12_smaller.pt"

model = YOLO(pathToModel)

Helpful functions, (similar to the one in `testPtFile.ipynb` inside `Prototyping/StandardTestOfModel/`)

In [ ]:
def gradeResults(result):
    predictions = []
    
    imgWidth = result[0].orig_shape[0]
    imgHeight = result[0].orig_shape[1]
    
    for box in result[0].boxes:
        predictedClass = model.names[int(box.cls)]
        confidence = round(float(box.conf), 3)
        
        x, y, w, h = box.xywh[0]
        x_offset = imgWidth/2 - (x)
        y_offset = imgHeight/2 - (y)
        offset = x_offset**2 + y_offset**2
        predictions.append((predictedClass, confidence, offset))
    # Outputs list of guesses, confidence, and offset from center.
    return predictions

def chooseTheMiddleOne(predictions):
    if len(predictions) == 0:
        return "empty", 0
    # Sorts by offset from center
    predictions.sort(key=lambda x: x[2])
    
    minAcceptableOffset = 400
    if predictions[0][2] > minAcceptableOffset**2:
        return "bad", 0
    
    MiddleBoxClass, confidence = predictions[0][:-1]
    return (MiddleBoxClass, confidence)

Making the predictions

In [ ]:
for building in SourceBuildings.values():
    for filename in building.filenames:
        ProcessedImage = model(filename)
        gradedPredictions = gradeResults(ProcessedImage)
        pred = chooseTheMiddleOne(gradedPredictions)
        building.addPrediction(filename, pred)
    building.MakePrediction()

In [ ]:
PrintStatistics(SourceBuildings)

In [ ]:
import json

def DumpResults(Buildings, postCode):
    DumpDictionary = {}
    for addr in Buildings:
      DumpDictionary[addr] = Buildings[addr].getDumpformat()
    with open(f'result_{postCode}.json', 'w') as fp:
        json.dump(DumpDictionary, fp)